<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/NN/Lab%201/Copy_of_Lab_1_Big_NN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Αϊδίνης Γιώργος 03116031

Κολιός Παναγιώτης 03116100

---

Ομάδα M.B.8

---


#Lab 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση - Μεγάλο Dataset (B10 - Epileptic Seizure Recognition)

In [0]:
!pip install --upgrade pip #upgrade pip package installer
!pip install scikit-learn --upgrade #upgrade scikit-learn package
!pip install numpy --upgrade #upgrade numpy package
!pip install pandas --upgrade #--upgrade #upgrade pandas package

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.22)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.17.4)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.25.3)


# Β. Εισαγωγή του Dataset

Multiclass Classification: Το συγκεκριμένο dataset χρησιμοποιείται και για multiclass και για binary (1 vs 2-5) classification. Λόγω του παραπάνω, και καθώς στο small dataset κάναμε binary classification επιλέξαμε να κάνουμε multiclass σε αυτό.

1. Έγινε καταγραφή της δραστηριότητας του εγκεφάλου 500 ατόμων για 23.6 δευτερόλεπτα/καταγραφή. Στόχος είναι η λήψη σωστής απόφασης περί του αν το άτομο βρίσκεται σε επιληπτική κρίση ή όχι. Έγινε δειγματοληψία κάθε καταγραφής, η οποία οδήγησε σε 4097 δείγματα. Τα 4097 δείγματα χωρίστηκαν σε 23 κομμάτια, με το καθένα να περιέχει 178 σημεία, που αντιστοιχούν σε 1 δευτερόλεπτο καταγραφής. Έτσι δημιουργήθηκαν 23*500 = 11500 γραμμές-δείγματα, καθένα από τα οποία αποτελείται από 178 σημεία που αντιστοιχούν σε 1 δευτερόλεπτο και αποτελόυν τη διάσταση των δεδομένων εισόδου. Η τελευταία στήλη περιέχει τις ετικέτες, οι οποίες παίρνουν τιμές 1-5, με την περίπτωση 1 να αφορά τις περιπτώσεις όπου το άτομο είχε επιληπτική κρίση και τις 2-5 τις περιπτώσεις όπου δεν είχε. 


In [0]:
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

In [2]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/00388/data.csv","ESR.csv")
print("All the files are downloaded")

Download file... ESR.csv ...
File downloaded
All the files are downloaded


In [3]:
!ls

ESR.csv  sample_data


In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("ESR.csv")

2. Όπως είπαμε παραπάνω και όπως φαίνεται παρακάτω, υπάρχουν 11500 δείγματα, το καθένα με 178 χαρακτηριστικά, τα οποία αφορούν τις μεταβολές της ηλεκτρικής τάσης των σημάτων στους νευρώνες του ανθρωπίνου εγκεφάλου. Συνεπώς είναι διατεταγμένα. Επίσης είναι ακέραιοι αριθμοί.
3. Υπάρχουν επικεφαλίδες στην πρώτη γραμμή και στοιχεία για το εκάστοτε δείγμα στην πρώτη στήλη, τα οποία θα πρέπει να αφαιρεθούν.
4. 

In [5]:
print(df.shape)

(11500, 180)


In [0]:
df #print the dataset

In [0]:
print ("We have ",len(df.columns), " attributes.")
for i in range(0, len(df.columns)):
    print('{:<10}{:<40}{:<10}{:<20}'.format(str(i+1), str(df.columns[i]),"type: ", str(df.dtypes[df.columns[i]])))

5. Διαγράφουμε την πρώτη κολώνα, ώστε όλες οι κολώνες εκτός της τελευταίας να περιέχουν τιμές των χαρακτηριστικών, με την τελευταία να περιέχει τις ετικέτες.

In [0]:
df = df.drop(df.columns[[0]], axis=1)
df

6. Δεν υπάρχουν απουσιάζουσες τιμές.
7. Χρησιμοποιούμε την bincount για να μετρήσουμε τη συχνότητα των κατηγοριών. Παρατηρούμε οτι έχουμε ενα εξαιρετικά ισορροπημένο dataset. Και οι 5 κατηγορίες είναι ισοπληθείς. 


In [9]:
labels = df.iloc[:, 178]
df = df.iloc[:, 0:178]   #remove lables from set
print("frequencies:", np.bincount(labels)[1:])

frequencies: [2300 2300 2300 2300 2300]


8. Διαχωρίζουμε σε train και test set. Οι τιμές των χαρακτηριστικών αφορούν τα εγκεφαλικά σήματα (τάσεις), άρα είναι διατεταγμένα.

In [10]:
from sklearn.model_selection import train_test_split

train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.3)
print(len(train))

8050


# Γ. Baseline Classification

##### Dummy Classifiers

Εκπαιδεύουμε τους classifiers με τις default τιμές για να δούμε συγκρίνουμε τα αποτέλεσματα πριν και μετά την προεργασία. <br>

Αρχίζουμε με τους Dummy Classifiers. Παρατηρούμε οτι επιτυγχάνουμε σε όλους περίπου 20% επιτυχία, όπως ήταν αναμενόμενο καθώς έχουμε 5 κατηγορίες.

In [19]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dc_uniform = DummyClassifier(strategy="uniform")
dc_constant_1 = DummyClassifier(strategy="constant", constant=1)
dc_constant_2 = DummyClassifier(strategy="constant", constant=2)
dc_most_frequent = DummyClassifier(strategy="most_frequent")
dc_stratified = DummyClassifier(strategy="stratified")

#using fit to train the classifiers
model_uniform = dc_uniform.fit(train, train_labels)
model_constant_1 = dc_constant_1.fit(train, train_labels)
model_constant_2 = dc_constant_2.fit(train, train_labels)
model_most_frequent = dc_most_frequent.fit(train, train_labels)
model_stratified = dc_stratified.fit(train, train_labels)

#now we make our predictions
preds_uniform = dc_uniform.predict(test)
preds_constant_1 = dc_constant_1.predict(test)
preds_constant_2 = dc_constant_2.predict(test)
preds_most_frequent = dc_most_frequent.predict(test)
preds_stratified = dc_stratified.predict(test)

#print prediction accuracy
accuracy = {}
print("Uniform Classifier: ", accuracy_score(test_labels, preds_uniform))
print("Constant Classifier (1): ", accuracy_score(test_labels, preds_constant_1))
print("Constant Classifier (2): ", accuracy_score(test_labels, preds_constant_2))
print("Most Frequent Classifier: ", accuracy_score(test_labels, preds_most_frequent))
print("Stratified Classifier: ", accuracy_score(test_labels, preds_stratified))

Uniform Classifier:  0.20492753623188406
Constant Classifier (1):  0.20115942028985506
Constant Classifier (2):  0.2046376811594203
Most Frequent Classifier:  0.196231884057971
Stratified Classifier:  0.20231884057971014


Εκτυπώνουμε τον confusion matrix. Ο $C$ είναι ο πίνακας για τον οποίο ισχύει οτι $C_{i, j}$ είναι τα δείγματα της κατηγορίας $i$ που ταξινομήθηκαν στην $j$. Όπως φαίνεται οι Uniform και Stratified προβλέπουν οποιαδήποτε κατηγορία, ενώ οι άλλοι 3 είτε επιλέγουν σταθερά μία κατηγορία είτε αυτή με τα περισσότερα δείγματα. Μάλιστα, από τη στιγμή που τα δείγματα στο αρχικό data set είναι ίσα μεταξύ τους, τα πολυπληθέστερα δείγματα στο train set θα είναι τα λιγότερο πολυπληθή στο test set. Άρα ο classifier αυτός θα έχει χειρότερη επίδοση από τους constant classifiers, όπως φαίνεται και από τον confusion matrix.

In [21]:
from sklearn.metrics import confusion_matrix

#produce confusion matrices
cnf_matrix_uniform = confusion_matrix(test_labels, preds_uniform)
cnf_matrix_constant_1 = confusion_matrix(test_labels, preds_constant_1)
cnf_matrix_constant_2 = confusion_matrix(test_labels, preds_constant_2)
cnf_matrix_most_frequent = confusion_matrix(test_labels, preds_most_frequent)
cnf_matrix_stratified = confusion_matrix(test_labels, preds_stratified)

#print confusion matrices
print("Uniform Classifier Confusion Matrix\n")
print(cnf_matrix_uniform, "\n")
print("Constant Classifier (1) Confusion Matrix\n")
print(cnf_matrix_constant_1, "\n")
print("Constant Classifier (2) Confusion Matrix\n")
print(cnf_matrix_constant_2, "\n")
print("Most Frequent Classifier Confusion Matrix\n")
print(cnf_matrix_most_frequent, "\n")
print("Stratified Classifier Confusion Matrix\n")
print(cnf_matrix_stratified, "\n")


Uniform Classifier Confusion Matrix

[[133 136 135 142 148]
 [169 146  96 144 151]
 [133 145 148 136 123]
 [143 131 118 155 130]
 [132 136 152 143 125]] 

Constant Classifier (1) Confusion Matrix

[[694   0   0   0   0]
 [706   0   0   0   0]
 [685   0   0   0   0]
 [677   0   0   0   0]
 [688   0   0   0   0]] 

Constant Classifier (2) Confusion Matrix

[[  0 694   0   0   0]
 [  0 706   0   0   0]
 [  0 685   0   0   0]
 [  0 677   0   0   0]
 [  0 688   0   0   0]] 

Most Frequent Classifier Confusion Matrix

[[  0   0   0 694   0]
 [  0   0   0 706   0]
 [  0   0   0 685   0]
 [  0   0   0 677   0]
 [  0   0   0 688   0]] 

Stratified Classifier Confusion Matrix

[[137 167 122 131 137]
 [140 135 146 147 138]
 [127 148 135 142 133]
 [139 142 128 145 123]
 [123 133 134 152 146]] 



Εκτύπωση f1-micro average και f1-macro average.

Η ακρίβεια αφορά την ικανότητα του εκτιμητή να εκτιμά ως δείγματα του test set που ανήκουν στη θετική κλάση μόνο αυτά που όντως ανήκουν.<br>
Precision: $$P = \frac{T_p}{T_p+F_p}$$

Η ανάκληση αφορά την ικανότητα του εκτιμητή να εντοπίζει όλα τα δείγματα που ανήκουν στη θετική κλάση.<br>
Recall: $$R = \frac{T_p}{T_p + F_n}$$

F1 score είναι ο αρμονικός μέσος όρος αυτών των δύο.<br>
F1: $$F1 = 2\frac{P \times R}{P+R}$$

Macro average: υπολογίζει f1 ξεχωριστά για κάθε κλάση και παίρνει τον μέσο όρο. Άρα κάθε κλάση αντιμετωπίζεται ισότιμα.

Micro average: ενσωματώνει την πληροφορία για τον αριθμό των δειγμάτων που ανήκουν σε κάθε κλάση, χρησιμοποιώντας τις πραγματικές ποσότητες $T_p$, $F_p$, $F_n$ στον συνολικό υπολογισμό. Άρα είναι προτιμότερη όταν μία κλάση περιέχει αρκετά περισσότερα δείγματα.

In [32]:
from sklearn.metrics import precision_recall_fscore_support
import warnings
warnings.filterwarnings("ignore")

#f1-micro
print('\033[1m' + "f1-micro" + '\033[0m')
print("Uniform Classifier: ", precision_recall_fscore_support(test_labels, preds_uniform, average='micro'))
print("Constant Classifier (1): ", precision_recall_fscore_support(test_labels, preds_constant_1, average='micro'))
print("Constant Classifier (2): ", precision_recall_fscore_support(test_labels, preds_constant_2, average='micro'))
print("Most Frequent Classifier: ", precision_recall_fscore_support(test_labels, preds_most_frequent, average='micro'))
print("Stratified Classifier: ", precision_recall_fscore_support(test_labels, preds_stratified, average='micro'), "\n")


#f1-macro
print('\033[1m' + "f1-macro" + '\033[0m')
print("Uniform Classifier: ", precision_recall_fscore_support(test_labels, preds_uniform, average='macro'))
print("Constant Classifier (1): ", precision_recall_fscore_support(test_labels, preds_constant_1, average='macro'))
print("Constant Classifier (2): ", precision_recall_fscore_support(test_labels, preds_constant_2, average='macro'))
print("Most Frequent Classifier: ", precision_recall_fscore_support(test_labels, preds_most_frequent, average='macro'))
print("Stratified Classifier: ", precision_recall_fscore_support(test_labels, preds_stratified, average='macro'))



f1-micro
Uniform Classifier:  (0.20492753623188406, 0.20492753623188406, 0.20492753623188406, None)
Constant Classifier (1):  (0.20115942028985506, 0.20115942028985506, 0.20115942028985506, None)
Constant Classifier (2):  (0.2046376811594203, 0.2046376811594203, 0.2046376811594203, None)
Most Frequent Classifier:  (0.196231884057971, 0.196231884057971, 0.196231884057971, None)
Stratified Classifier:  (0.20231884057971014, 0.20231884057971014, 0.20231884057971014, None) 

f1-macro
Uniform Classifier:  (0.20513152276246868, 0.20502744287274183, 0.20499468736487234, None)
Constant Classifier (1):  (0.04023188405797101, 0.2, 0.06698841698841698, None)
Constant Classifier (2):  (0.04092753623188406, 0.2, 0.06794995187680461, None)
Most Frequent Classifier:  (0.0392463768115942, 0.2, 0.06561667070511266, None)
Stratified Classifier:  (0.2025617905893406, 0.20241885429206366, 0.20242073614464723, None)


##### Gaussian Naive Bayes

Ο Gaussian Nauve Bayes υποθέτει οτι τα χαρακτηριστικά είναι ανεξάρτητα μεταξύ τους. Έτσι συνδέει κάθε χαρακτηριστικό $x_i$ με κάθε κλάση $y$ με την πιθανότητα $P(x_i \mid y)$, η οποία υποθέτει πως ακολουθεί γκαουσιανή κατανομή. Χρησιμοποιεί τα δεδομένα προκειμένου για κάθε κλάση και χαρακτηριστικό, το οποίο παίρνει συνεχείς τιμές, να βρει τη μέση τιμή $\mu_y$ και τη διακύμανση $\sigma^2_y$ κάθε χαρακτηριστικού για τη κλάση $y$. Στην φάση του testing λαμβάνει υπόψην του τα γινόμενα των παραπάνω πιθανοτήτων των χαρακτηριστικών για κάθε κλάση, καθώς και την πιθανότητα της ίδιας της κλάσης και αναθέτει στο δείγμα την κλάση που μεγιστοποιεί το τελικό γινόμενο.

Παρατηρούμε πως η ενσωμάτωση πληροφορίας, ακόμα και όταν έχουμε κάνει τις παραπάνω υποθέσεις υπερδιπλασιάζει την ακρίβεια των προβλέψεών μας.

Επίσης, όπως φαίνεται από το confusion matrix υπάρχει αναθέτει πολλά δείγματα στην πέμπτη κλάση. Ωστόσο επιτυγχάνει σε μεγάλο βαθμό να εντοπίσει τις επιληπτικές κρίσεις (πρώτη κλάση).

In [25]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

#finding mean values and variations
model_GNB = gnb.fit(train, train_labels)

#making predictions
preds = {}
preds["Gaussian Naive Bayes"] = gnb.predict(test)

#calculate and print prediction accuracy
accuracy = {}
accuracy["Gaussian Naive Bayes"] = accuracy_score(test_labels, preds["Gaussian Naive Bayes"])
print("Gaussian Naive Bayes: ", accuracy["Gaussian Naive Bayes"], "\n")

#produce and print confusion matrix
cnf_matrix_GNB = confusion_matrix(test_labels, preds["Gaussian Naive Bayes"])
print('\033[1m' + "Gaussian Naive Bayes - Confusion Matrix" + '\033[0m')
print(cnf_matrix_GNB, "\n")

#f1-micro/macro
print('\033[1m' + "f1-micro" + '\033[0m')
print("Gaussian Naive Bayes: ", precision_recall_fscore_support(test_labels, preds["Gaussian Naive Bayes"], average='micro'))
print('\033[1m' + "f1-macro" + '\033[0m')
print("Gaussian Naive Bayes: ", precision_recall_fscore_support(test_labels, preds["Gaussian Naive Bayes"], average='macro'))



Gaussian Naive Bayes:  0.4350724637681159 

Gaussian Naive Bayes - Confusion Matrix
[[571 122   0   1   0]
 [ 36 120 100  80 370]
 [  2  89 116 115 363]
 [  0 172 141 186 178]
 [  0  28  94  58 508]] 

f1-micro
Gaussian Naive Bayes:  (0.4350724637681159, 0.4350724637681159, 0.435072463768116, None)
f1-macro
Gaussian Naive Bayes:  (0.440304679906499, 0.43503898147138953, 0.41798384443582093, None)


##### kNN

Ο kNN υπολογίζει, για κάθε δείγμα του test set, τους k κοντινότερους γείτονές του, οι οποίοι είναι δείγματα του train set, στον n-διάστατο χώρο διαστάσεων των χαρακτηριστικών εισόδου. Αποφασίζει την κλάση του νέου δείγματα παίρνοντας είτε την πλειοψηφία των γειτόνων είτε λαμβάνοντας υπόψην και τις αποστάσεις του από αυτούς. Ως συνάρτηση απόστασης χρησιμοποιείται κυρίως η ευκλείδια. <br>
Εξαιρετικά σημαντική για την απόδοσή του είναι η υπερπαράμετρος k. 

Το γεγονός οτι πρέπει να συγκρίνουμε την απόσταση κάθε νέου δείγματος στο train set με το νέο δείγμα καθιστά τη διαδικασία πρόβλεψης πολύ χρονοβόρα.
Ωστόσο, η μεγάλη διαφορά από τον GNB είναι οτι δεν κάνει υποθέσεις για τις εξαρτήσεις των χαρακτηριστικών και τις κατανομές των πιθανοτήτων.

Παρατηρούμε οτι επιτυγχάνει καλύτερες προβλέψεις από τον kNN, ωστόσο το ποσοστό είναι ακόμα χαμηλό.

In [28]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5) #setting k to 5

#saves training samples and their labels
knn.fit(train, train_labels)

#making predictions
preds["kNN"] = knn.predict(test)

#calculate and print prediction accuracy
accuracy["kNN"] = accuracy_score(test_labels, preds["kNN"])
print("kNN: ", accuracy["kNN"])

kNN:  0.4692753623188406


In [29]:
#produce and print confusion matrix
cnf_matrix_GNB = confusion_matrix(test_labels, preds["kNN"])
print('\033[1m' + "kNN - Confusion Matrix" + '\033[0m')
print(cnf_matrix_GNB, "\n")

#f1-micro/macro
print('\033[1m' + "f1-micro" + '\033[0m')
print("kNN: ", precision_recall_fscore_support(test_labels, preds["kNN"], average='micro'))
print('\033[1m' + "f1-macro" + '\033[0m')
print("kNN: ", precision_recall_fscore_support(test_labels, preds["kNN"], average='macro'))

kNN - Confusion Matrix
[[474 102  80  32   6]
 [  3 444 249   1   9]
 [  0 231 448   0   6]
 [  0 180 191 197 109]
 [  0 285 343   4  56]] 

f1-micro
kNN:  (0.4692753623188406, 0.4692753623188406, 0.4692753623188406, None)
f1-macro
kNN:  (0.5671756200262825, 0.4676583819869663, 0.4549905293668945, None)


##### Multi Layer Perceptron

Παρατηρούμε οτι δεν αποδίδει καλύτερα ούτε από τους Dummy Classifiers.

In [31]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5,), random_state=1)

#training
clf.fit(train, train_labels)

#predicting
preds["Multi Layer Perceptron"] = clf.predict(test)

#calculate and print prediction accuracy
accuracy["Multi Layer Perceptron"] = accuracy_score(test_labels, preds["Multi Layer Perceptron"])
print("Multi Layer Perceptron: ", accuracy["Multi Layer Perceptron"])

#produce and print confusion matrix
cnf_matrix_GNB = confusion_matrix(test_labels, preds["Multi Layer Perceptron"])
print('\033[1m' + "Multi Layer Perceptron - Confusion Matrix" + '\033[0m')
print(cnf_matrix_GNB, "\n")

#f1-micro/macro
print('\033[1m' + "f1-micro" + '\033[0m')
print("Multi Layer Perceptron: ", precision_recall_fscore_support(test_labels, preds["Multi Layer Perceptron"], average='micro'))
print('\033[1m' + "f1-macro" + '\033[0m')
print("Multi Layer Perceptron: ", precision_recall_fscore_support(test_labels, preds["Multi Layer Perceptron"], average='macro'))

Multi Layer Perceptron:  0.19652173913043477
Multi Layer Perceptron - Confusion Matrix
[[190 282  36 142  44]
 [213 298   7 169  19]
 [187 280  13 184  21]
 [180 305  24 152  16]
 [208 284  23 148  25]] 

f1-micro
Multi Layer Perceptron:  (0.19652173913043477, 0.19652173913043477, 0.19652173913043475, None)
f1-macro
Multi Layer Perceptron:  (0.18346833292812365, 0.19514135716533865, 0.1609712261114407, None)


# Δ. Βελτιστοποίηση Ταξινομητών

Χρησιμοποιούμε την διαδικασία pipeline για την εφαρμογή διαδοχικών μετασχηματισμών στα χαρακτηριστικά των δεδομένων. 

* Λόγω του ισορροπημένου dataset δεν χρειάζεται να χρησιμοποιήσουμε oversampler ή undersampler.

* Διαστατικότητα: Χρησιμοποιούμε τον μετασχηματιστή VarianceThreshold, ο οποίος μειώνει τον αριθμό των χαρακτηριστικών με βάση την διακύμανση των τιμών του στα δείγματα (επιλογή χαρακτηριστικών). Όταν η διακύμανση είναι μικρή θεωρούμε οτι το χαρακτηριστικό δεν προσφέρει πολλή πληροφορία για την κατηγοριοποίηση. <br>
Επίσης χρησιμοποιούμε τον PCA για την ανάλυση των δεδομένων σε κύριες συνιστώσες και την χρήση των συνιστωσών με την περισσότερη διακύμανση (εξαγωγή χαρακτηριστικών), δηλαδή πληροφορία. 

* Κανονικοποίηση: Αμβλύνουμε τις διαφορές μεταξύ των τιμών των χαρακτηριστικών. Αν ένα χαρακτηριστικό έχει πολύ μεγαλύτερες τιμές από ένα άλλο η σημασία του σε εκτιμητές όπως ο kNN, που μετρά τις αποστάσεις από τα χαρακτηριστικά, είναι μεγαλύτερη χωρίς αυτό να σημαίνει οτι παρέχει περισσότερη πληροφορία για την κατηγοριοποίηση. Χρησιμοποιούμε δύο μετασχηματιστές κανονικοποίησης, τον scaler και τον min_max_scaler. 

Η σειρά που ακολουθείται είναι η εξής:
0. minmax αν VarianceThreshold για να μην επηρεαστεί η επιλογή από τις τιμές των χαρακτηριστικών
1. Κανονικοποίηση (minmax ή z-score)
2. PCA
3. Εκτιμητής



Χρήση grid search για την βελτιστοποίηση των υπερπαραμέτρων. Φτιάχνουμε σύνολο πιθανών συνδυασμών τιμών των παραμέτρων για να βρόυμε τον βέλτιστο. Υπολογίζεται ο μέσος όρος σε όλα τα folds (5 εδώ) του cross-validation με βάση της f1 μετρικές. 